# College Football DraftKings Lineup Simulation and Optimizer

This is step 3 of college football fantasy projections process. After data scraping and processing, I read data from .csv files (projections and ownership). This data is used for a linear programming problem (similar to knapsack problem) where the goal is to maximize points under several constraints (salary, position, team).

**Future Improvements**
There are too many moving parts - the goal is to get all data scraping, analysis, simulation, etc. into one place. The API scraping and linear programming steps are best completed in Python, so converting the R code to Python is a sensible next step. Also, code documentation can be improved.

The ultimate goal is to automate all steps: scrape historical player stats, run point projection models, perform simulations, generate *n* optimal lineups, upload to DraftKings

In [1]:
# For data frames
import pandas as pd

# For generating random number from normal distribution
import numpy as np

# from platform import python_version

# print(python_version())
import pulp

import numpy as np
from sklearn.linear_model import LinearRegression

In [88]:
# Read in projections .csv file for slate
slate_projections = pd.read_csv("SimulatedProjections_Week7.csv")
slate_projections.head(10)

,Player,Team,Proj_Points,index,Position,overUnder,Salary
0,Abdur-Rahmaan Yaseen,Purdue,2.232138,2143,WR,0.0,3000
1,Abdur-Rahmaan Yaseen,Purdue,-1.862707,3349,WR,0.0,3000
2,Abdur-Rahmaan Yaseen,Purdue,4.092737,6431,WR,0.0,3000
3,Abdur-Rahmaan Yaseen,Purdue,4.987144,3146,WR,0.0,3000
4,Abdur-Rahmaan Yaseen,Purdue,-3.270813,917,WR,0.0,3000
5,Abdur-Rahmaan Yaseen,Purdue,4.900583,2867,WR,0.0,3000
6,Abdur-Rahmaan Yaseen,Purdue,-0.582325,595,WR,0.0,3000
7,Abdur-Rahmaan Yaseen,Purdue,-0.120240,8576,WR,0.0,3000
8,Abdur-Rahmaan Yaseen,Purdue,-0.236709,9467,WR,0.0,3000
9,Abdur-Rahmaan Yaseen,Purdue,-2.025212,8357,WR,0.0,3000


In [8]:
slate_projections = pd.read_csv("DK NCAAF Ownership.csv")
slate_projections.head(10)

,Player,Team,Proj_Points,Position,Salary,Opponent,Ownership%
0,Jahcour Pearson,Ole Miss,10.34,WR,3200,LSU,38.4
1,Jerome Ford,Cincinnati,26.08,RB,8300,Navy,37.7
2,Travis Dye,Oregon,25.57,RB,7400,UCLA,30.3
3,Matt Corral,Ole Miss,30.34,QB,10200,LSU,30.1
4,Deuce Vaughn,Kansas St,25.18,RB,7700,Texas Tech,30.0
5,Breece Hall,Iowa State,28.66,RB,9400,Okla State,25.5
6,Jahan Dotson,Penn State,19.62,WR,7200,Illinois,22.8
7,Jahan Dotson,Penn State,19.62,WR,7200,Illinois,22.8
8,Caleb Williams,Oklahoma,37.60,QB,9800,Kansas,19.6
9,Max Johnson,LSU,22.41,QB,7200,Ole Miss,19.2


In [9]:
# Add indicator variable for position (use in optimizer)
slate_projections['QB'] = [1 if x.strip() == 'QB' else 0 for x in slate_projections['Position']]
slate_projections['RB'] = [1 if x.strip() == 'RB' else 0 for x in slate_projections['Position']]
slate_projections['WR'] = [1 if x.strip() == 'WR' else 0 for x in slate_projections['Position']]

slate_projections.head(10)

,Player,Team,Proj_Points,Position,Salary,Opponent,Ownership%,QB,RB,WR
0,Jahcour Pearson,Ole Miss,10.34,WR,3200,LSU,38.4,0,0,1
1,Jerome Ford,Cincinnati,26.08,RB,8300,Navy,37.7,0,1,0
2,Travis Dye,Oregon,25.57,RB,7400,UCLA,30.3,0,1,0
3,Matt Corral,Ole Miss,30.34,QB,10200,LSU,30.1,1,0,0
4,Deuce Vaughn,Kansas St,25.18,RB,7700,Texas Tech,30.0,0,1,0
5,Breece Hall,Iowa State,28.66,RB,9400,Okla State,25.5,0,1,0
6,Jahan Dotson,Penn State,19.62,WR,7200,Illinois,22.8,0,0,1
7,Jahan Dotson,Penn State,19.62,WR,7200,Illinois,22.8,0,0,1
8,Caleb Williams,Oklahoma,37.60,QB,9800,Kansas,19.6,1,0,0
9,Max Johnson,LSU,22.41,QB,7200,Ole Miss,19.2,1,0,0


In [10]:
def lineup_optimizer(df):
    
    # Define pulp object
    model = pulp.LpProblem('CFB', pulp.LpMaximize)

    # Initialize dicts that will store parameters. Keys are decision variables below 
    total_points = {}
    cost = {}
    QBs = {}
    RBs = {}
    WRs = {}
    number_of_players = {}

    for i, player in df.iterrows():
        var_name = 'x' + str(i) # Create variable name
        decision_var = pulp.LpVariable(var_name, cat='Binary') # Initialize Variables

        total_points[decision_var] = player["Proj_Points"] # Create PPG Dictionary
        cost[decision_var] = player["Salary"] # Create Cost Dictionary

        # Create Dictionary for Player Types
        QBs[decision_var] = player["QB"]
        RBs[decision_var] = player["RB"]
        WRs[decision_var] = player["WR"]
        number_of_players[decision_var] = 1.0

    # Define objective function and add it to the model
    objective_function = pulp.LpAffineExpression(total_points)
    model += objective_function
    
    #Define cost constraint and add it to the model
    total_cost = pulp.LpAffineExpression(cost)
    model += (total_cost <= 50000)

    # Add player type constraints
    QB_constraint = pulp.LpAffineExpression(QBs)
    RB_constraint = pulp.LpAffineExpression(RBs)
    WR_constraint = pulp.LpAffineExpression(WRs)
    total_players = pulp.LpAffineExpression(number_of_players)

    model += (QB_constraint >= 1)
    model += (QB_constraint <= 2) # Can have 'super flex'
    model += (RB_constraint >= 2)
    model += (RB_constraint <= 4) # 2 starters + flex + super flex
    model += (WR_constraint >= 3)
    model += (WR_constraint <= 5) # 3 starters + flex + super flex
    model += (total_players <= 8)

    #model.status
    model.solve()
    
    # Add variable to original data frame: = 1 if player is drafted, = 0 if not drafted
    df['is_drafted'] = 0.0
    for var in model.variables():
        # Set is drafted to the value determined by the LP
        df.iloc[int(var.name[1:]),10] = var.varValue # column index 10 = is_drafted
        
    # Save optimal team in a new df
    optimal_team = df[df["is_drafted"] == 1.0]
    
    return(optimal_team)

In [11]:
optimal = lineup_optimizer(slate_projections)


,Player,Team,Proj_Points,Position,Salary,Opponent,Ownership%,QB,RB,WR,is_drafted
0,Jahcour Pearson,Ole Miss,10.34,WR,3200,LSU,38.4,0,0,1,1.0
2,Travis Dye,Oregon,25.57,RB,7400,UCLA,30.3,0,1,0,1.0
4,Deuce Vaughn,Kansas St,25.18,RB,7700,Texas Tech,30.0,0,1,0,1.0
8,Caleb Williams,Oklahoma,37.60,QB,9800,Kansas,19.6,1,0,0,1.0
13,Blake Corum,Michigan,23.80,RB,7100,Nwestern,17.2,0,1,0,1.0
19,Brian Thomas Jr.,LSU,10.69,WR,4000,Ole Miss,13.9,0,0,1,1.0
21,Malik Nabers,LSU,10.79,WR,3700,Ole Miss,12.8,0,0,1,1.0
24,Sean Clifford,Penn State,25.51,QB,6900,Illinois,11.3,1,0,0,1.0


In [91]:
# This function changes position to desired Draftkings output

def get_DK_pos(row):
    if row['newPos'] == 'QB1':
        return 'QB'
    elif row['newPos'] =='QB2':
        return 'S-FLEX'
#     elif row['newPos'] =='RB1':
#         return 'RB'
#     elif row['newPos'] =='RB2':
#         return 'RB'
    elif row['newPos'] =='RB3':
        return 'FLEX'
    elif row['newPos'] =='RB4':
        return 'S-FLEX'
#     elif row['newPos'] =='WR1':
#         return 'WR'
#     elif row['newPos'] =='WR2':
#         return 'WR'
#     elif row['newPos'] =='WR3':
#         return 'WR'
    elif row['newPos'] =='WR4':
        return 'FLEX'
    elif row['newPos'] =='WR5':
        return 'S-FLEX'
    else:
        return row['newPos']

In [92]:
# For mapping positions to DK positions
df_mapping = pd.DataFrame({
    'Pos': ['QB', 'RB1', 'RB2', 'WR1', 'WR2', 'WR3', 'FLEX', 'S-FLEX'],
})
sort_mapping = df_mapping.reset_index().set_index('Pos')

# Optimal lineup for n lineups
n = 1000
n_optimals = pd.DataFrame(columns=['Player','Team','Position','Proj_Points','Salary'])
for i in range(1, n+1):
    df = slate_projections.loc[slate_projections['index']==i].reset_index(drop=True)
    optimal_lineup_i = lineup_optimizer(df)
    optimal_lineup_i = optimal_lineup_i[['Player','Team','Position','Proj_Points','Salary']]
    
    # Order by position for DraftKings output
    optimal_lineup_i['cumsum'] = (optimal_lineup_i.groupby(['Position']).cumcount() + 1).astype(str)
    optimal_lineup_i['newPos'] = optimal_lineup_i['Position'] + optimal_lineup_i['cumsum']
    
    optimal_lineup_i['combo'] = optimal_lineup_i.apply(get_DK_pos, axis=1)
    # order by updated position
    optimal_lineup_i['size_num'] = optimal_lineup_i['combo'].map(sort_mapping['index'])
    optimal_lineup_i = optimal_lineup_i.sort_values('size_num')
    optimal_lineup_i['stripped'] = optimal_lineup_i['combo'].str.replace('\d+', '')
    
    # Drop columns
    optimal_lineup_i.drop(columns=['newPos', 'cumsum', 'size_num', 'combo'], inplace = True)
    
    # Set index
    optimal_lineup_i['nth lineup'] = i
    
    n_optimals = n_optimals.append(optimal_lineup_i)
    
    if(i%1000==0):
        print(i, "Sequencing the mainframe...")
        if(i%10000==0):
            print("mmmmmmmmmmmmm")
#     salary = sum(optimal_lineup_i['Salary'])
#     points = sum(optimal_lineup_i['Proj_Points'])
    
#     print(optimal_lineup_i, "\n\n", "Points:", points, " Salary:", salary, "\n\n")
    #print(i, ":", optimal_lineup_i)

1000 Sequencing the mainframe...


In [93]:
# 10,000 lineups takes about 18-20 minutes

# Count times each player shows up in optimal lineup
counts = n_optimals.groupby(['Player', 'Salary', 'Position']).size()

# convert to df
counts_df = counts.to_frame(name = 'NumLineups').reset_index()

# Calculate probability player is in optimal lineup
counts_df['OptimalPercent'] = counts_df['NumLineups'] / n

# Sort by optimal percentage
counts_df = counts_df.sort_values(by = 'OptimalPercent', ascending = False).reset_index(drop = True)
print(counts_df)
counts_df.to_csv(r'C:\Users\punco\OneDrive\Desktop\Fantasy Football\2021\CFB\Percentage_CFBWeek7.csv', index=False)

                Player  Salary Position  NumLineups  OptimalPercent
0     Mar'Keise Irving    3900       RB         907           0.907
1     Charleston Rambo    6100       WR         424           0.424
2       Tyler Van Dyke    6900       QB         368           0.368
3    Wan'Dale Robinson    6400       WR         346           0.346
4        Tyler Goodson    6500       RB         341           0.341
..                 ...     ...      ...         ...             ...
166     Jacolby Hewitt    3200       WR           1           0.001
167      Tanner Morgan    5400       QB           1           0.001
168  Marcus Washington    3000       WR           1           0.001
169      Jackson Meeks    3000       WR           1           0.001
170    Adonai Mitchell    3400       WR           1           0.001

[171 rows x 5 columns]


In [94]:
# Count frequencies of each lineup combination
n_optimals['idx'] = (n_optimals.groupby(['nth lineup']).cumcount() + 1).astype(str)

# Sort players by nth lineup then alphabetically
n_optimals.sort_values(["nth lineup"], ascending = (True))

n_optimals_wide = (n_optimals.pivot_table(index=['nth lineup'], 
                      columns=['idx'], 
                      values=['Player', 'Proj_Points'], 
                      aggfunc='first'))

# Total points by summing columns
n_optimals_wide.to_csv(r'C:\Users\punco\OneDrive\Desktop\Fantasy Football\2021\CFB\OptimalsCFBWeek7.csv', index=False)

#n_optimals_wide['TotalPoints'] = sum(n_optimals_wide.loc['Proj_Points'])
n_optimals_wide.head(10)

Player                                          \
idx                          1                   2                   3   
nth lineup                                                               
1.0             Connor Bazelak      Cam'Ron Harris    Mar'Keise Irving   
2.0                 Sam Howell    Mar'Keise Irving         Tyler Badie   
3.0                 Sam Howell    Mar'Keise Irving       Tyler Goodson   
4.0               Zach Calzada       Jaylen Warren  Kenneth Walker III   
5.0             Desmond Ridder  Kenneth Walker III    Mar'Keise Irving   
6.0             Connor Bazelak      Bijan Robinson       Jaylen Warren   
7.0             Connor Bazelak    Mar'Keise Irving       Trestan Ebner   
8.0             Connor Bazelak       Jaylen Warren  Kenneth Walker III   
9.0              Kenny Pickett      Isaiah Spiller    Mar'Keise Irving   
10.0        Braxton Burmeister      Bijan Robinson  Kenneth Walker III   

                                                                     \
idx                        4                   5                  6   
nth lineup                                                            
1.0         Charleston Rambo  Demetris Robertson         Josh Downs   
2.0               Tre Turner         Ty Fryfogle    Tyquan Thornton   
3.0           Camron Buckley          Josh Downs        Ty Fryfogle   
4.0               Tre Turner     Tyquan Thornton  Wan'Dale Robinson   
5.0         Charleston Rambo  Demetris Robertson        Ty Fryfogle   
6.0         Charleston Rambo         Deion Smith        Ty Fryfogle   
7.0         Charleston Rambo         Mike Harley  Wan'Dale Robinson   
8.0          Jackson Anthrop          Josh Downs        Ty Fryfogle   
9.0              Deion Smith         Mike Harley        Ty Fryfogle   
10.0         Brennan Presley       Treylon Burks  Wan'Dale Robinson   

                                               Proj_Points             \
idx                          7               8           1          2   
nth lineup                                                              
1.0              Tyler Goodson  Tyler Van Dyke   24.315763  22.711010   
2.0              Tyler Goodson  Tyler Van Dyke   31.548414  19.878768   
3.0            Tyquan Thornton  Tyler Van Dyke   33.426072  21.975899   
4.0           Mar'Keise Irving     Tyler Badie   25.312036  24.543866   
5.0              Tyler Goodson    Zach Calzada   27.522505  27.660129   
6.0           Mar'Keise Irving  Tyler Van Dyke   27.518614  26.698962   
7.0                Tyler Badie    Zach Calzada   28.317293  22.428846   
8.0         Tyrion Davis-Price     Max Johnson   29.246833  28.165696   
9.0                Tyler Badie     Max Johnson   34.579044  23.278318   
10.0          Mar'Keise Irving  Connor Bazelak   26.058257  28.245346   

                                                                              
idx                 3          4          5          6          7          8  
nth lineup                                                                    
1.0         23.106046  19.551773  18.747847  18.310262  22.331894  24.096689  
2.0         29.467906  18.412929  21.977344  17.887560  21.056083  26.783764  
3.0         22.170355  15.799439  25.020614  19.795584  19.626001  27.404016  
4.0         25.137383  19.675476  21.425542  20.300768  24.170192  26.991300  
5.0         21.599379  21.647172  18.902470  20.210001  26.477267  24.171092  
6.0         25.182019  20.093094  17.581137  22.557535  22.472181  27.341661  
7.0         20.589894  17.810129  24.233228  19.129025  28.679857  28.979689  
8.0         27.585671  15.881505  25.512564  18.354368  21.878900  27.735777  
9.0         19.666782  17.246865  20.469613  27.099228  26.567764  32.257903  
10.0        24.177235  15.019854  21.321599  18.224923  21.137819  23.989330

In [95]:
# Concatenate each player in nth lineup to create lineup_ID; use to count frequency of lineup combos
cols = ['1', '2', '3', '4', '5', '6', '7', '8']

n_optimals_wide['LineupID'] = n_optimals_wide['Player'][cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
lineup_strings = pd.DataFrame(n_optimals_wide['LineupID'].value_counts())
lineup_strings.reset_index(level=0, inplace=True)
string_counts = n_optimals_wide.merge(lineup_strings, how="left", left_on = "LineupID", right_on = "index")
string_counts.sort_values(by = 'LineupID', ascending = False).reset_index(drop = True).drop_duplicates()

C:\Users\punco\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


,"(Player, 1)","(Player, 2)","(Player, 3)","(Player, 4)","(Player, 5)","(Player, 6)","(Player, 7)","(Player, 8)","(Proj_Points, 1)","(Proj_Points, 2)","(Proj_Points, 3)","(Proj_Points, 4)","(Proj_Points, 5)","(Proj_Points, 6)","(Proj_Points, 7)","(Proj_Points, 8)","(LineupID, )",index,LineupID
0,Bo Nix,Kenneth Walker III,Mar'Keise Irving,Charleston Rambo,Mike Harley,Wan'Dale Robinson,Tyler Badie,Connor Bazelak,25.600574,22.285111,22.978178,17.325413,17.306700,20.619010,24.344639,25.080632,Bo Nix_Kenneth Walker III_Mar'Keise Irving_Cha...,Bo Nix_Kenneth Walker III_Mar'Keise Irving_Cha...,2
1,Max Johnson,Jerome Ford,Mar'Keise Irving,Charleston Rambo,Treylon Burks,Tyquan Thornton,Tyler Goodson,Zach Calzada,25.906004,24.493983,27.164570,20.762617,19.100997,21.143251,23.569215,26.381527,Max Johnson_Jerome Ford_Mar'Keise Irving_Charl...,Max Johnson_Jerome Ford_Mar'Keise Irving_Charl...,2
2,Sam Howell,Mar'Keise Irving,Tyler Badie,Tre Turner,Ty Fryfogle,Tyquan Thornton,Tyler Goodson,Tyler Van Dyke,31.548414,19.878768,29.467906,18.412929,21.977344,17.887560,21.056083,26.783764,Sam Howell_Mar'Keise Irving_Tyler Badie_Tre Tu...,Sam Howell_Mar'Keise Irving_Tyler Badie_Tre Tu...,2
3,Bo Nix,Kenneth Walker III,Mar'Keise Irving,Charleston Rambo,Mike Harley,Wan'Dale Robinson,Tyler Badie,Connor Bazelak,19.861323,26.776384,21.893012,18.893148,19.270228,22.152464,26.154161,28.274649,Bo Nix_Kenneth Walker III_Mar'Keise Irving_Cha...,Bo Nix_Kenneth Walker III_Mar'Keise Irving_Cha...,2
4,Max Johnson,Jerome Ford,Mar'Keise Irving,Charleston Rambo,Treylon Burks,Tyquan Thornton,Tyler Goodson,Zach Calzada,27.895892,20.731983,26.081976,17.605761,19.587870,27.079634,23.714293,23.751290,Max Johnson_Jerome Ford_Mar'Keise Irving_Charl...,Max Johnson_Jerome Ford_Mar'Keise Irving_Charl...,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Connor Bazelak,Jaylen Warren,Mar'Keise Irving,Charleston Rambo,Jackson Anthrop,Mike Harley,Tyler Badie,Tyler Van Dyke,26.270977,21.135442,22.856383,19.149994,16.788343,20.497522,23.683273,26.110056,Connor Bazelak_Jaylen Warren_Mar'Keise Irving_...,Connor Bazelak_Jaylen Warren_Mar'Keise Irving_...,1
996,Connor Bazelak,Bijan Robinson,Mar'Keise Irving,Chance Luper,Charleston Rambo,Mike Harley,Tyler Goodson,Tyler Van Dyke,27.246760,30.155662,21.229965,15.062089,20.799382,20.012149,25.650894,27.933375,Connor Bazelak_Bijan Robinson_Mar'Keise Irving...,Connor Bazelak_Bijan Robinson_Mar'Keise Irving...,1
997,Max Johnson,Bijan Robinson,Mar'Keise Irving,Demetris Robertson,Josh Downs,Warren Thompson,Raheim Sanders,Sam Howell,25.859724,30.857428,25.325680,16.851661,28.021914,12.612754,16.904101,31.378764,Max Johnson_Bijan Robinson_Mar'Keise Irving_De...,Max Johnson_Bijan Robinson_Mar'Keise Irving_De...,1
998,Connor Bazelak,Jaylen Warren,Mar'Keise Irving,Brennan Presley,Jackson Anthrop,Josh Downs,Tyler Badie,Max Johnson,24.845178,22.605637,24.317818,16.750614,14.875379,24.574161,24.063400,27.984089,Connor Bazelak_Jaylen Warren_Mar'Keise Irving_...,Connor Bazelak_Jaylen Warren_Mar'Keise Irving_...,1


In [9]:
# Figure out how to 'score' lineups based on correlation, ownership (leverage), etc.

In [10]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 18:21:24
